In [26]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [27]:
data = pd.read_csv('data/train.csv')

In [28]:
data.shape

(7613, 5)

In [29]:
data['target'].value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [30]:
data.drop_duplicates(subset=['text'], inplace=True ,keep=False, ignore_index=True)

In [31]:
data.shape

(7434, 5)

In [32]:
exemple = data.text[40]
exemple

"on the outside you're ablaze and alive\nbut you're dead inside"

In [33]:
X = data['text']
y = data['target']

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

In [35]:
seed =42

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [37]:
model = make_pipeline

625     The #IranDeal only covers nuclear activity. Wh...
4525    Inundated with employee holiday request paperw...
101     'The man who can drive himself further once th...
2492           My Chemical Romance ÛÓ Desolation Row #np
2020    17 dead as Afghanistan aircraft crashes: An Af...
                              ...                        
751                   I blew up snapchat for no reason ??
3290    VIETNAM WAR PATCH US 71st EVACUATION HOSPITAL ...
1600    Catching up on GBBO and omg that girls cake th...
5271    @ushiocomics I may be panicking a little I was...
2206    Businesses are deluged with invoices. Make you...
Name: text, Length: 1487, dtype: object